In [ ]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

import sys, os
sys.path.append("<path>")

from tqdm.auto import tqdm 
from pprint import pprint
import pandas as pd 

import load_llms 

In [ ]:
from load_llms import load_llm_in_4bit
'''
Locally stored LLM paths
'''
llm_paths = {
}

In [ ]:
def prep_data():
    prompts = pd.read_csv("<path>")[['prompt']]
    return prompts 

In [ ]:
def create_prompt(x):
    
    instruction = '''Pretend you are a medical patient. Write a message to your doctor using the message prompt. 
    
    ### Rules ###
    - Assume the doctor you are messaging has been your physician for years. It is permissable to speak informally when appropriate. 
    - Do not restate the prompt in the message.
    - You may add additional health context (e.g. symptoms or medications) to the message as needed. 
    '''

    chat = []
    ### Add test sample at the end ###
    chat.append({'role': 'user', 'content': f'''{instruction}
    
        Prompt: {x['prompt']}

        Patient Message:'''})
    
    return chat 

In [ ]:
### Parse the pipeline outputs ### 
def parse_outputs(x):
    return x[0]['generated_text'][-1]['content']
def parse_prompts(x):
    return x[-1]['content']

In [ ]:
import gc 
def run(model_str):
    
    pipe, tokenizer, model = load_llms.load_llm_in_4bit(llm_paths[model_str])
    prompts = prep_data()
    
    ### Shuffle inputs before each generation ### 
    all_prompts = [create_prompt(prompts.iloc[x]) for x in range(len(prompts))]
    outputs = pipe(all_prompts, temperature = 0.75, do_sample=True, batch_size = 6)
    parsed_outputs = [{'message':parse_outputs(x), 'prompt':parse_prompts(p)} for x,p in zip(outputs, all_prompts)]
    
    pd.DataFrame(parsed_outputs).to_csv("<path>")
    
    ### Clean up models to load another ### 
    pipe = None
    model = None 
    gc.collect()
    torch.cuda.empty_cache()